*UE Learning from User-generated Data, CP MMS, JKU Linz 2022*
# Exercise 2: Collaborative Filtering + Implicit Feedback

In this exercise we create our first proper recommender: Item-based Collaborative Filtering on implicit feedback. Please refer to the slides and recording of the Second UE session published on Moodle if you have any problems.

The assignment submission deadline is 05.04.2022 12:00.

Make sure to rename the notebook according to the convention:\
LUD22_ex01_k<font color='red'>\<Matr. Number\></font>_<font color='red'>\<Surname-Name\></font>.ipynb

for example:

LUD22_ex01_k000007_Bond-James.ipynb

## Implementation
In this exercise you are reqired to write three functions each calling the previous. Every function will be graded separately. Insert your implementations into the templates provided. Please don't change the templates even I they are not pretty. Don't forget to test your implementation for correctness and efficiency (a single run of any function should not take more than 2 minutes).

Please only use libraries already imported in the notebook. **Feel free to experiment with the notebook, but clean it up before submitting.**

There is also a bonus task which won't earn you any points on its own. However it can help earn bonus points while solving analysis tasks on Moodle (will be published later).

## Item-Based Collaborative Filtering
The idea of Item-Based Collaborative Filtering is to estimate if a user **u** is going to like item **i** throught checking how similar this item is to the items already consumed (and rated) by the user. We calculate the estimation as a sum of ratings given by **u** to the consumed items weighted with their respective similarities to the item **i**. Please note, that from all items consumed by the user we only consider top **N** items most similiar to item **i**.

In case of implicit feedback (which we deal with in this exercise), all "ratings" are either 0 or 1. Therefore we combine our score from the similarities themselves. In this exercise we take the average of similarities to top **N** (or less if not enough neighbors was found) most similar to **i** items. Also note that we don't have to account for biases and missing values.


In [1]:
import pandas as pd
import numpy as np

## Interaction Matrix

In this exercise we work with the same format of interaction matrices as in the Exercise 1 (default settings) and the same data. Find a way to use the matrix in this notebook (e.g. copy your implementation of inter_matr_binary here and create the matrix anew).

In [2]:
def inter_matr_binary(usr_path = 'sampled_1000_items_demo.txt',
                      itm_path = 'sampled_1000_items_tracks.txt',
                      inter_path = 'sampled_1000_items_inter.txt',
                      threshold = 1)  -> np.ndarray:
    '''
    usr_path - string path to the file with users data;
    itm_path - string path to the file with item data;
    inter_path - string path to the file with interaction data;
    threshold - int > 0, criteria of a valid interaction
    
    returns - 2D np.array, rows - users, columns - items;
    '''
    
    # Read files
    usr = pd.read_csv(usr_path, sep="\t", header=None).values
    itm = pd.read_csv(itm_path, sep="\t", header=None).values
    inter = pd.read_csv(inter_path, sep="\t", header=None).values
    
    # Create interaction matrix
    res = np.zeros(shape=(len(usr), len(itm)))
    for interaction in inter:
        res[interaction[0], interaction[1]] = int(interaction[2] >= threshold)
    
    return res

In [3]:
path = ''

usr_path = path + 'sampled_1000_items_demo.txt'
itm_path = path + 'sampled_1000_items_tracks.txt'
inter_path = path + 'sampled_1000_items_inter.txt'

_interaction_matrix_test = inter_matr_binary(usr_path, itm_path, inter_path)
_interaction_matrix_test

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

## <font color='red'>TASK 1/3</font>: Get item similarities
This is a helper function to be later used for user-item score estimation.
The function should take two arguments: a binary interaction_matrix-like numpy array **inter** and a plane binary vector corresponding to an item **target_vec**. You can expect that the length of the vector corresponds to the number of users in the matrix **inter** (first parameter).

*the vector can be just a slice of the interaction matrix, see asserts*

Expected output: array of similarities between every item in **inter** and the vector **target_vec**. The similarities need to be placed in the order the items appear in the martrix **inter**.

Example: **inter** is a 7 by 3 matrix, containing information about 3 items and 7 users (can be expressed through item vectors as [it1; it2; it3]). **target_vec** is a vector of length 7 (assuming it tells us about interactions between the item and the same 7 users). The expected output is an array of length 3:\
[*sim*(it1, target_vec), *sim*(it2, target_vec), *sim*(it3, target_vec)]

**Similarity:** use jaccard score as the similarity measure. Please implement it yourself, don't use any external libraries;\
If $a$ and $b$ are two items, let's define $U(a)$ as the set of users, interacted with the item $a$ (same for $b$). $|U(a)|$ corresponds to the number of users interacted with the item $a$. Then Jaccard similarity score between the two items is defined as:
$JaccardScore(a,b) = \frac{|U(a) \wedge U(b)|}{|U(a) \vee U(b)|}$

In words: Jaccard Score is the number of users interacted with both items divided by the number of users interacted with at least one of them.

Use the cell below to define the similarity as a helper function, please don't change the parameters, name or output format.

<b>Tip:</b> The item vectors are on the axis=1 in the matrix.<br>

In [4]:
from sklearn.metrics import jaccard_score

In [5]:
def my_jaccard_score(a: np.array, b: np.array) -> float:
    """
    a, b: - vectors of the same length corresponding to the two items

    returns: float - jaccard similarity score for a and b
    """
    
    score = None
    
    # TODO: YOUR IMPLEMENTATION.
    
    score = np.sum(np.sum(np.logical_and(a, b)))/np.sum(np.logical_or(a, b))
    
    return score

In [6]:
def calculate_sim_scores(inter: np.array, target_vec: np.array) -> np.array:
    """
    inter: np.array - interaction matrix - calculate similarity between each item and the target item (see below)
    target_vec: int - target item vector
    
    use my_jaccard_similarity function from above
    
    returns: np.array - similarities between every item from <inter> and <target_vec> in the respective order
    """
    
    item_similarities = None
    
    # TODO: YOUR IMPLEMENTATION.
    
    item_similarities = [my_jaccard_score(item, target_vec) for item in inter.T]
    
    return np.array(item_similarities)

Now please call the function for the <b>whole</b> _interaction_matrix_test and the vector of the item with the <b>id 0</b>.

In [7]:
item_sims = None

# TODO: YOUR IMPLEMENTATION

item_sims = calculate_sim_scores(_interaction_matrix_test, _interaction_matrix_test[:, 0]) 

print(item_sims)

[1.         0.02083333 0.01030928 0.03529412 0.01449275 0.05555556
 0.04347826 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.04545455 0.
 0.         0.04545455 0.11764706 0.05882353 0.05882353 0.02150538
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.01898734 0.         0.         0.04545455 0.
 0.         0.         0.         0.01265823 0.         0.
 0.         0.         0.0625     0.         0.         0.05
 0.05555556 0.05555556 0.08695652 0.05769231 0.0625     0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.07142857 0.         0.         0.
 0.0625     0.01315789 0.0625     0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.04       0.         0.         0.         0.         0.03174603
 0.         0.         0.01234

In [8]:
assert item_sims is not None, "item_sims should not be None."
assert type(item_sims) == np.ndarray, "types are not correct."
assert len(item_sims) == 412, "length is not correct."
assert item_sims[0] == 1, "Item at the index 0 should have sim of 1"

## <font color='red'>TASK 2/3</font>: Estimate user-item score

Write a function that takes a full interaction matrix as an input, as well as user id, item id and *n* -- algorithm's hyperparameter, number of neighbors to be considered while calculating the score.

The expected output is a single number between 0 and 1 - the predicted score.

Refer to the slides and the recording. Follow the algorithm:
* take items consumed by the user
* calculate the similarities between them and the target item (exclude the target user from consideration when calculating the similarities!)
* return average of top **n** highest similarity scores

<b>Tip:</b> Copy the interaction matrix before using it.<br>

In [9]:
def get_user_item_score(inter: np.array, 
                            target_user: int, 
                            target_item: int, 
                            n: int = 2) -> float:
    """
    inter: np.array - interaction matrix.
    target_user: target user id
    target_item: int - target item id
    n: int - n closest neighbors to consider for the score prediction
    
    returns: float - mean of similarity scores
    """
    
    item_similarities_mean = None
    
    # TODO: YOUR IMPLEMENTATION.
    
    consumed_items = inter.T[np.where(inter.T[:, target_user] == 1)].T
    # remove target_user element
    consumed_items = np.delete(consumed_items, target_user, axis=0)
    inter_wo_user = np.delete(inter, target_user, axis=0)
    
    sim_scores = calculate_sim_scores(consumed_items, inter_wo_user[:, target_item])
    # divide by len(sim_scores) if there are less consumed elements then n 
    item_similarities_mean = np.sum(np.sort(sim_scores)[-n:] / min(len(sim_scores), n))
        
    return item_similarities_mean
    

Now, call your function for <b>user 0</b> and <b>item 0</b> and <b> n = 10</b>

In [10]:
item_sim = None

# TODO: YOUR IMPLEMENTATION.

item_sim = get_user_item_score(_interaction_matrix_test, 0, 0, n = 10) 

print(item_sim)

0.14776226279985677


In [11]:
assert item_sim is not None, "item sim should have a value."
assert item_sim <= 1 and item_sim >= 0, "value of item sim is not valid."

## <font color='red'>TASK 3/3</font>: Recommender

Implement the recommender function for the scoring algorithm you implemented above.
The function takes a full interaction matrix, user id, top_k, hyperparameter **n** as an input. It returns two arrays: top_k recommendations for the given user with the algorithm using given number of **n** neighbors for score prediction and the corresponding scores.

Make sure you recommend items the user hasn't seen before! Try to optimizing your implementation so that the runs take seconds rather than minutes.

Please don't change the "interface" of the function.

In [1]:
from sklearn.metrics import pairwise_distances

In [12]:
def recTopK(inter_matr: np.array,
            user: int,
            top_k: int,
            n: int) -> (np.array, np.array):
    '''
    inter_matr - np.array from the task 1
    user - user_id, integer
    top_k - expected length of the resulting list
    n - number of neighbors to consider
    
    returns - array of recommendations (sorted in the order of descending scores) & array of corresponding scores
    '''
    
    top_rec = None
    scores = None
    
    # TODO: YOUR IMPLEMENTATION.
    
    all_scores = list()
    
    for item_id in range(len(inter_matr.T)):
        if inter_matr.T[item_id, user] == 1:  # skip consumed items
            continue
        all_scores.append([item_id, get_user_item_score(inter_matr, user, item_id, n)])
    
    # sort
    all_scores.sort(key=lambda x: x[1])
    all_scores.reverse()
    # take top_k
    all_scores = np.array(all_scores[:top_k])
    # split in recommended items and scores
    top_rec = all_scores[:, 0].astype(int)
    scores = all_scores[:, 1]
    
    return np.array(top_rec), np.array(scores)

Now, lets use these scoring functions and get the <b>top 10</b> recommendations for <b>user 0</b> with <b>n = 15</b>.

In [13]:
rec_item_cf = None
scores_item_cf = None

# TODO: YOUR IMPLEMENTATION

rec_item_cf, scores_item_cf = recTopK(_interaction_matrix_test, 0, top_k = 10, n=15)

In [14]:
print("Recommendations with Item CF: ", rec_item_cf)
print("With Scores: ", scores_item_cf)
print("-" * 75)

Recommendations with Item CF:  [117  51  56  12  43 129  30 167  98   8]
With Scores:  [0.05742293 0.05148129 0.05079365 0.0505204  0.04872741 0.04715545
 0.04533492 0.04288674 0.04026317 0.03986823]
---------------------------------------------------------------------------


In [15]:
assert rec_item_cf is not None, "Recommendations should not be None."
assert type(rec_item_cf) == np.ndarray, "Types should be np.ndarray."
assert len(rec_item_cf) == 10, "10 recommendations should be returned."

In [16]:
assert scores_item_cf is not None, "Scores should not be None."
assert type(scores_item_cf) == np.ndarray, "Types should be np.ndarray."
assert len(scores_item_cf) == 10, "10 recommendations should be returned."

# [BONUS] Get Track Info
This task won't be graded, you don't have to submit any functions. Make sure to have specified implementations at hand if you plan to try earn bonus points in some analysis tasks later during the course.

In [17]:
# Write a function which receives a list 
# of item IDs and path to the file with Track information 
# and returns a list of strings (Artist-Track)

def get_artist_track(item_ids, track_file):
    itm = pd.read_csv(track_file, sep="\t", header=None).values
    return [str(itm[item_id][0])+' - '+str(itm[item_id][1]) for item_id in item_ids]

In [18]:
# Leave this cell the way it is, please.